In [13]:
import torch
import torch_tensorrt
from pytorch_quantization import quant_modules
from models.experimental import attempt_load
from utils.utils_qat import *
from utils.datasets import create_dataloader
import yaml
from utils.general import colorstr
import copy

In [14]:
w = './runs/train/yolov7-tiny/weights/best.pt'
# Load model
quant_modules.initialize() # QAT
torch_model = attempt_load(w).eval().cuda()
torch_model.model[-1].training = True

Fusing layers... 
IDetect.fuse


In [15]:
torch_model(torch.rand((1,3,384, 672)).cuda())

fuseforward training
fuseforward training
fuseforward training


(tensor([[[[[-4.87547e-01,  9.47899e-02,  5.22560e-01,  ..., -6.50625e+00, -6.23370e+00, -4.29142e+00],
            [-2.87736e-01, -1.69391e-01,  7.51416e-01,  ..., -6.16169e+00, -5.12955e+00, -3.47082e+00],
            [-5.59410e-01, -1.65552e-02,  9.41929e-01,  ..., -6.76621e+00, -5.27977e+00, -4.36671e+00],
            ...,
            [-8.52922e-01, -1.31474e-02,  7.39594e-01,  ..., -7.51798e+00, -5.87583e+00, -4.87661e+00],
            [-3.33085e-01,  9.42448e-03,  7.67701e-01,  ..., -8.03420e+00, -6.72067e+00, -5.64604e+00],
            [-8.26194e-01,  8.08707e-02,  1.12668e+00,  ..., -5.60131e+00, -5.30826e+00, -3.10908e+00]],
 
           [[-2.08714e-01, -1.34601e+00,  6.99600e-01,  ..., -4.64982e+00, -4.97741e+00, -2.42952e+00],
            [-4.18270e-01, -6.20992e-01,  1.00484e+00,  ..., -5.67229e+00, -4.35944e+00, -3.12736e+00],
            [ 1.66784e-02, -6.98056e-01,  9.36961e-01,  ..., -6.31265e+00, -5.18270e+00, -4.00719e+00],
            ...,
            [-1.08629e+00, 

In [16]:
torch_model

Model(
  (model): Sequential(
    (0): Conv(
      (conv): QuantConv2d(
        3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)
        (_input_quantizer): TensorQuantizer(8bit fake per-tensor amax=dynamic calibrator=MaxCalibrator scale=1.0 quant)
        (_weight_quantizer): TensorQuantizer(8bit fake axis=0 amax=dynamic calibrator=MaxCalibrator scale=1.0 quant)
      )
      (act): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (1): Conv(
      (conv): QuantConv2d(
        32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)
        (_input_quantizer): TensorQuantizer(8bit fake per-tensor amax=dynamic calibrator=MaxCalibrator scale=1.0 quant)
        (_weight_quantizer): TensorQuantizer(8bit fake axis=0 amax=dynamic calibrator=MaxCalibrator scale=1.0 quant)
      )
      (act): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (2): Conv(
      (conv): QuantConv2d(
        64, 32, kernel_size=(1, 1), stride=(1, 1)
        (_input_quantizer): TensorQuantizer(8bi

In [17]:
torch_model(torch.rand((1,3,384,672)).cuda())

fuseforward training
fuseforward training
fuseforward training


(tensor([[[[[-4.13326e-01, -6.23371e-02,  5.75005e-01,  ..., -5.31502e+00, -5.16107e+00, -2.58336e+00],
            [-1.75843e-01, -2.12433e-01,  8.34009e-01,  ..., -6.20939e+00, -5.19469e+00, -3.07918e+00],
            [-4.22990e-01, -1.98272e-01,  8.13441e-01,  ..., -6.89601e+00, -5.63713e+00, -3.84209e+00],
            ...,
            [ 1.32598e-01,  5.98902e-02,  9.51860e-01,  ..., -7.06737e+00, -5.26251e+00, -4.33658e+00],
            [-7.16299e-01, -4.58673e-02,  1.16375e+00,  ..., -7.49755e+00, -6.05372e+00, -5.10794e+00],
            [ 3.38462e-02, -1.87414e-02,  8.11759e-01,  ..., -6.72266e+00, -6.11831e+00, -3.99034e+00]],
 
           [[-2.16347e-01, -7.94555e-01,  5.66289e-01,  ..., -5.38766e+00, -5.03973e+00, -2.49367e+00],
            [-3.46139e-01, -3.42304e-01,  9.74601e-01,  ..., -5.99159e+00, -4.85145e+00, -3.14609e+00],
            [-7.45145e-01,  6.87071e-02,  9.42374e-01,  ..., -5.85254e+00, -4.92627e+00, -3.48064e+00],
            ...,
            [-6.90832e-01, 

In [18]:
# For dataset
imgsz = 640
batch_size = 1
gs = int(max(torch_model.stride))  # grid size (max stride)

with open('data/hyp.scratch.tiny.yaml') as f:
    hyp = yaml.load(f, Loader=yaml.SafeLoader)  # load hyps
device='cuda'

with open('./data/VisDrone.yaml') as f:
    data_dict = yaml.load(f, Loader=yaml.SafeLoader)  # data dict
    
train_path = data_dict['train']
test_path = data_dict['test']
dataloader, dataset = create_dataloader(train_path, imgsz, batch_size, gs, opt=None,
                                    hyp=hyp, augment=True, prefix=colorstr('train: '))

train: Scanning '../datasets/VisDrone/VisDrone2019-DET-train/labels.cache' images and labels... 6471 found, 0 missing, 0 empty, 4 corrupted: 100%|█| 6471/6471 


In [19]:
qat_model = torch_model.cuda()
modified_state_dict = {}
for key, val in qat_model.state_dict().items():

    if key.startswith('module'):
        modified_state_dict[key[7:]] = val
    else:
        modified_state_dict[key] = val

qat_model.load_state_dict(modified_state_dict)

<All keys matched successfully>

In [20]:
torch_model(torch.rand((1,3,384,672)).cuda())

fuseforward training
fuseforward training
fuseforward training


(tensor([[[[[-3.03100e-01,  8.63281e-03,  4.69530e-01,  ..., -5.78400e+00, -4.84398e+00, -3.23647e+00],
            [-2.85701e-01, -8.69404e-02,  5.23454e-01,  ..., -7.12912e+00, -6.14127e+00, -4.98199e+00],
            [-1.25635e+00, -5.33806e-02,  8.93788e-01,  ..., -6.50298e+00, -5.29050e+00, -3.29647e+00],
            ...,
            [-4.42532e-02,  2.11913e-02,  8.73779e-01,  ..., -7.55342e+00, -5.99426e+00, -5.13522e+00],
            [-2.63270e-01, -1.73550e-02,  1.09284e+00,  ..., -7.57492e+00, -6.28809e+00, -5.00204e+00],
            [-5.91200e-01, -2.54331e-02,  1.12599e+00,  ..., -6.36567e+00, -6.07072e+00, -4.09629e+00]],
 
           [[-2.66897e-01, -8.99185e-02,  6.64318e-01,  ..., -5.55964e+00, -5.34481e+00, -3.88217e+00],
            [-8.72903e-01, -3.89067e-01,  7.57212e-01,  ..., -6.28030e+00, -5.19728e+00, -4.38327e+00],
            [-2.53817e-01, -3.04515e-01,  7.93453e-01,  ..., -6.85807e+00, -5.34068e+00, -4.34973e+00],
            ...,
            [-1.54081e-01, 

In [21]:
with torch.no_grad():
    calibrate_model(
        model=qat_model,
        model_name='yolov7-tiny',
        data_loader=dataloader,
        num_calib_batch=32,
        calibrator="max",
        hist_percentile=[99.9, 99.99, 99.999, 99.9999],
        out_dir="./runs/train/yolov7-tiny/")

Calibrating model


  3%|███▊                                                                                                                       | 1/32 [00:00<00:03,  7.88it/s]

fuseforward training
fuseforward training
fuseforward training


  6%|███████▋                                                                                                                   | 2/32 [00:00<00:03,  7.83it/s]

fuseforward training
fuseforward training
fuseforward training


...

fuseforward training
fuseforward training
fuseforward training
fuseforward training
fuseforward training
fuseforward training
fuseforward training
fuseforward training
fuseforward training
fuseforward training
fuseforward training
fuseforward training


W0615 14:21:07.299917 140418879964992 tensor_quantizer.py:173] Disable MaxCalibrator
W0615 14:21:07.300198 140418879964992 tensor_quantizer.py:173] Disable MaxCalibrator
W0615 14:21:07.300467 140418879964992 tensor_quantizer.py:173] Disable MaxCalibrator
W0615 14:21:07.300720 140418879964992 tensor_quantizer.py:173] Disable MaxCalibrator
W0615 14:21:07.301927 140418879964992 tensor_quantizer.py:173] Disable MaxCalibrator
W0615 14:21:07.302227 140418879964992 tensor_quantizer.py:173] Disable MaxCalibrator
W0615 14:21:07.302485 140418879964992 tensor_quantizer.py:173] Disable MaxCalibrator
W0615 14:21:07.302745 140418879964992 tensor_quantizer.py:173] Disable MaxCalibrator
W0615 14:21:07.302993 140418879964992 tensor_quantizer.py:173] Disable MaxCalibrator
W0615 14:21:07.303253 140418879964992 tensor_quantizer.py:173] Disable MaxCalibrator
W0615 14:21:07.303500 140418879964992 tensor_quantizer.py:173] Disable MaxCalibrator
W0615 14:21:07.303754 140418879964992 tensor_quantizer.py:173] Di

W0615 14:21:07.325731 140418879964992 tensor_quantizer.py:173] Disable MaxCalibrator
W0615 14:21:07.325976 140418879964992 tensor_quantizer.py:173] Disable MaxCalibrator
W0615 14:21:07.326229 140418879964992 tensor_quantizer.py:173] Disable MaxCalibrator
W0615 14:21:07.326488 140418879964992 tensor_quantizer.py:173] Disable MaxCalibrator
W0615 14:21:07.326907 140418879964992 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0615 14:21:07.333152 140418879964992 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([32, 1, 1, 1]).
W0615 14:21:07.333743 140418879964992 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0615 14:21:07.334409 140418879964992 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([64, 1, 1, 1]).
W0615 14:21:07.334939 140418879964992 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0615 14:21:07.335577 140418879964992 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([

W0615 14:21:07.376688 140418879964992 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0615 14:21:07.377484 140418879964992 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([32, 1, 1, 1]).
W0615 14:21:07.378035 140418879964992 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0615 14:21:07.378716 140418879964992 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([32, 1, 1, 1]).
W0615 14:21:07.379264 140418879964992 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0615 14:21:07.380019 140418879964992 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([32, 1, 1, 1]).
W0615 14:21:07.380566 140418879964992 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0615 14:21:07.381316 140418879964992 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([32, 1, 1, 1]).
W0615 14:21:07.381878 140418879964992 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Siz

Skip quantization: 
Skip quantization: model
Skip quantization: model.0
Skip quantization: model.0.conv
model.0.conv._input_quantizer           : TensorQuantizer(8bit fake per-tensor amax=1.0000 calibrator=MaxCalibrator scale=1.0 quant)
model.0.conv._weight_quantizer          : TensorQuantizer(8bit fake axis=0 amax=[0.8467, 9.8950](32) calibrator=MaxCalibrator scale=1.0 quant)
Skip quantization: model.0.act
Skip quantization: model.1
Skip quantization: model.1.conv
model.1.conv._input_quantizer           : TensorQuantizer(8bit fake per-tensor amax=21.9565 calibrator=MaxCalibrator scale=1.0 quant)
model.1.conv._weight_quantizer          : TensorQuantizer(8bit fake axis=0 amax=[0.1028, 1.0065](64) calibrator=MaxCalibrator scale=1.0 quant)
Skip quantization: model.1.act
Skip quantization: model.2
Skip quantization: model.2.conv
model.2.conv._input_quantizer           : TensorQuantizer(8bit fake per-tensor amax=31.6012 calibrator=MaxCalibrator scale=1.0 quant)
model.2.conv._weight_quantize

In [22]:
qat_model(torch.rand((1,3,640,640)).cuda())

fuseforward training
fuseforward training
fuseforward training


(tensor([[[[[-4.80892e-01,  2.16270e-01,  4.86674e-01,  ..., -5.82189e+00, -5.50311e+00, -2.94533e+00],
            [-6.06903e-01, -4.96320e-02,  5.14759e-01,  ..., -7.16952e+00, -5.98059e+00, -4.31843e+00],
            [-7.81228e-01,  9.20628e-02,  4.61599e-01,  ..., -7.55950e+00, -6.04208e+00, -4.39546e+00],
            ...,
            [-8.87838e-02, -1.49041e-01,  8.43424e-01,  ..., -7.36928e+00, -6.51112e+00, -5.47578e+00],
            [-1.29246e+00, -1.56100e-01,  1.27963e+00,  ..., -6.00930e+00, -5.14863e+00, -3.37378e+00],
            [-3.33980e-01,  1.59899e-01,  1.14406e+00,  ..., -4.98250e+00, -4.58480e+00, -2.78846e+00]],
 
           [[-3.28819e-01, -5.15180e-02,  5.64076e-01,  ..., -5.64666e+00, -5.58934e+00, -3.09178e+00],
            [-9.41898e-01, -8.29980e-01,  6.91706e-01,  ..., -5.99931e+00, -4.94950e+00, -3.50759e+00],
            [-2.51756e-01, -1.09771e+00,  5.05238e-01,  ..., -7.64555e+00, -5.64651e+00, -4.33606e+00],
            ...,
            [-3.04189e-01, 

In [23]:
quant_nn.TensorQuantizer.use_fb_fake_quant = True
with torch.no_grad():
    data = iter(dataloader)
    images, _,_,_ = next(data)
    images = (images/255.0).cuda()
    jit_model = torch.jit.trace(qat_model, images.to("cuda"))
    torch.jit.save(jit_model, "trained_vgg16_qat.jit.pt")

E0615 14:21:07.571968 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:07.572519 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:07.574281 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:07.574583 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:07.579425 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:07.579825 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:07.581447 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:07.581789 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:07.586458 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale expl

E0615 14:21:07.701926 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:07.702162 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:07.705913 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:07.706160 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:07.707381 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:07.707617 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:07.711303 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:07.711558 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:07.712790 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale expl

E0615 14:21:07.814574 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:07.814785 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:07.815910 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:07.816124 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:07.819688 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:07.819947 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:07.821134 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:07.821339 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:07.825651 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale expl

E0615 14:21:08.049036 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.049306 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.053107 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.053398 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.054484 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.054757 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.058460 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.058743 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.059817 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale expl

E0615 14:21:08.155211 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.155464 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.156790 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.157036 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.160909 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.161161 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.162359 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.162604 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.166823 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale expl

fuseforward training
fuseforward training
fuseforward training


E0615 14:21:08.211385 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.211788 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.213011 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.213257 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.216838 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.217093 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.218309 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.218553 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.222132 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale expl

E0615 14:21:08.315640 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.315844 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.320095 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.320371 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.321533 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.321779 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.325308 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.325578 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale explicitly!
E0615 14:21:08.326616 140418879964992 tensor_quantizer.py:120] Fake quantize mode doesn't use scale expl

fuseforward training
fuseforward training
fuseforward training
fuseforward training
fuseforward training
fuseforward training


In [24]:
torch_model.cpu().eval()

Model(
  (model): Sequential(
    (0): Conv(
      (conv): QuantConv2d(
        3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)
        (_input_quantizer): TensorQuantizer(8bit fake per-tensor amax=1.0000 calibrator=MaxCalibrator scale=1.0 quant)
        (_weight_quantizer): TensorQuantizer(8bit fake axis=0 amax=[0.8467, 9.8950](32) calibrator=MaxCalibrator scale=1.0 quant)
      )
      (act): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (1): Conv(
      (conv): QuantConv2d(
        32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)
        (_input_quantizer): TensorQuantizer(8bit fake per-tensor amax=21.9565 calibrator=MaxCalibrator scale=1.0 quant)
        (_weight_quantizer): TensorQuantizer(8bit fake axis=0 amax=[0.1028, 1.0065](64) calibrator=MaxCalibrator scale=1.0 quant)
      )
      (act): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (2): Conv(
      (conv): QuantConv2d(
        64, 32, kernel_size=(1, 1), stride=(1, 1)
        (_input_quanti

In [25]:
torch_model(torch.rand((1,3,384,672)).cpu())

(tensor([[[2.63411e+00, 5.58906e+00, 4.83296e+00,  ..., 1.07195e-03, 2.33448e-03, 7.97935e-03],
          [7.66330e+00, 4.77671e+00, 5.89252e+00,  ..., 1.31260e-03, 4.92726e-03, 1.87256e-02],
          [1.82627e+01, 4.07512e+00, 5.74930e+00,  ..., 7.25613e-04, 2.68547e-03, 1.27729e-02],
          ...,
          [5.98103e+02, 3.75296e+02, 3.74372e+01,  ..., 7.77785e-03, 1.46844e-02, 8.82231e-03],
          [6.20645e+02, 3.76648e+02, 5.15256e+01,  ..., 1.02470e-02, 3.01174e-02, 9.34405e-03],
          [6.49215e+02, 3.71123e+02, 4.15049e+01,  ..., 1.01309e-02, 2.29706e-02, 2.25935e-02]]]),
 [tensor([[[[[-3.44849e-01,  4.02616e-01,  4.85726e-01,  ..., -6.83720e+00, -6.05763e+00, -4.82289e+00],
             [-1.21421e+00,  1.94792e-01,  7.70379e-01,  ..., -6.63443e+00, -5.30803e+00, -3.95896e+00],
             [-4.41349e-01,  1.87811e-02,  7.32000e-01,  ..., -7.22777e+00, -5.91721e+00, -4.34757e+00],
             ...,
             [-6.40948e-01, -2.28585e-02,  4.00428e-01,  ..., -8.06027e+0

In [26]:
qat_model = torch.jit.load("trained_vgg16_qat.jit.pt").eval()
compile_spec = {"inputs": [torch_tensorrt.Input([1, 3, 384, 672])],
                "enabled_precisions": torch.int8,
                "truncate_long_and_double": True,
                }
trt_mod = torch_tensorrt.compile(qat_model, **compile_spec)


In [34]:
input_image = torch.rand((1,3,384,672)).cuda()
o1 = qat_model(input_image)
# o2 = jit_model(input_image)
o3 = trt_mod(input_image)

In [28]:
# np.testing.assert_almost_equal(o1[0].cpu().numpy(),o3[0].cpu().numpy())
# np.testing.assert_almost_equal(o1[1].cpu().numpy(),o2[1].cpu().numpy())
# np.testing.assert_almost_equal(o1[2].cpu().numpy(),o2[2].cpu().numpy())

In [35]:
torch.jit.save(trt_mod, w.replace('.pt', '_qat.trt'))

In [36]:
trt_mod = torch.jit.load(w.replace('.pt', '_qat.trt'))

In [40]:
trt_mod(input_image)

(tensor([[[[[-2.22109e-01,  1.50951e-01,  5.21803e-01,  ..., -5.93185e+00, -5.16824e+00, -2.88184e+00],
            [-5.07687e-01,  7.32737e-02,  6.19502e-01,  ..., -6.55571e+00, -5.13715e+00, -3.13503e+00],
            [-2.99645e-01,  1.55214e-01,  5.13571e-01,  ..., -7.34297e+00, -5.36995e+00, -4.05769e+00],
            ...,
            [-3.03014e-01,  2.90084e-02,  6.87270e-01,  ..., -7.07415e+00, -5.00878e+00, -4.26948e+00],
            [-3.77576e-01,  1.02184e-01,  7.82256e-01,  ..., -7.54790e+00, -6.13167e+00, -5.22417e+00],
            [-3.26356e-01, -1.61818e-01,  8.72771e-01,  ..., -5.59588e+00, -5.10294e+00, -3.18662e+00]],
 
           [[-1.73041e-01, -9.29553e-01,  2.76176e-01,  ..., -6.00224e+00, -5.54974e+00, -3.42740e+00],
            [-6.80190e-01, -1.48092e-01,  5.99334e-01,  ..., -7.14898e+00, -5.58389e+00, -4.51367e+00],
            [-3.38186e-01, -4.56995e-01,  6.28369e-01,  ..., -7.10099e+00, -5.24368e+00, -4.00169e+00],
            ...,
            [-3.99954e-01, 

In [33]:
from test import pseudo_inference
ci = pseudo_inference(w)

Fusing layers... 
IDetect.fuse


In [37]:
B,C,H,W = input_image.shape

In [46]:
torch_model.eval().cpu()
t1, t2 = torch_model(input_image.cpu())

In [47]:
o1, o2 = ci.run(trt_mod(input_image), nx_list=[W//8, W//16, W//32], ny_list=[H//8, H//16, H//32])

In [48]:
t1

tensor([[[2.48691e+00, 3.49555e+00, 5.12630e+00,  ..., 8.08980e-04, 7.84201e-03, 1.27847e-01],
         [1.04869e+01, 3.49555e+00, 5.12630e+00,  ..., 8.08980e-04, 7.84201e-03, 1.27847e-01],
         [1.84869e+01, 3.49555e+00, 5.12630e+00,  ..., 8.08980e-04, 7.84201e-03, 1.27847e-01],
         ...,
         [5.97028e+02, 3.53377e+02, 6.12005e+01,  ..., 1.01705e-02, 1.50065e-03, 3.06086e-03],
         [6.15579e+02, 3.57104e+02, 5.20234e+01,  ..., 4.78162e-03, 2.56205e-03, 4.58687e-03],
         [6.37331e+02, 3.67696e+02, 4.92063e+01,  ..., 6.70077e-03, 1.62391e-02, 9.34259e-03]]])

In [49]:
o1

tensor([[[2.48691e+00, 3.49555e+00, 5.12630e+00,  ..., 8.08982e-04, 7.84202e-03, 1.27847e-01],
         [1.04869e+01, 3.49555e+00, 5.12630e+00,  ..., 8.08982e-04, 7.84202e-03, 1.27847e-01],
         [1.84869e+01, 3.49555e+00, 5.12630e+00,  ..., 8.08982e-04, 7.84202e-03, 1.27847e-01],
         ...,
         [5.90872e+02, 3.74057e+02, 2.90891e+01,  ..., 2.20489e-03, 8.76354e-04, 8.86529e-04],
         [6.23391e+02, 3.70629e+02, 3.82358e+01,  ..., 5.86631e-04, 4.67911e-04, 5.00718e-04],
         [6.42960e+02, 3.75359e+02, 3.29330e+01,  ..., 2.74538e-03, 1.64567e-03, 2.05125e-03]]], device='cuda:0')

### Test 

In [15]:
from utils.general import non_max_suppression, scale_coords, check_img_size
from pathlib import Path
from utils.loss import ComputeLoss
from torch.cuda import amp

In [26]:
torch_model.eval().cuda()
compute_loss = ComputeLoss(torch_model)  # init loss class

In [32]:
save_hybrid = False
cuda = 'cuda'
device='cuda'
conf_thres = 1e-4
iou_thres = 0.6
seen = 0
scaler = amp.GradScaler(enabled=cuda)
gs = max(int(torch_model.stride.max()), 32)  # grid size (max stride)
imgsz, imgsz_test = [check_img_size(x, gs) for x in [640, 640]]  # verify imgsz are gs-multiples
# test_path = data_dict['test']

# train_loader = create_dataloader(test_path, imgsz_test, batch_size * 2, gs, opt=None,  # testloader
#                                        hyp=hyp, cache=None, rect=True, rank=-1,
#                                        world_size=1, workers=8,
#                                        pad=0.5, prefix=colorstr('val: '))[0]

for batch_i, (img, targets, paths, shapes) in enumerate(tqdm(dataloader, desc='test')):
    nb, _, height, width = img.shape  # batch size, channels, height, width
    img = img.to(device, non_blocking=True).float()/255.0
    out, train_out = torch_model(img)  # forward
    
    targets[:, 2:] *= torch.Tensor([width, height, width, height]).to('cpu')  # to pixels
    lb = [targets[targets[:, 0] == i, 1:] for i in range(nb)] if save_hybrid else []  # for autolabelling
    out = non_max_suppression(out, conf_thres=conf_thres, iou_thres=iou_thres, labels=lb, multi_label=True)
    print(loss)
    print(scaler.scale(loss))
    break

test:   0%|                                                                                                                           | 0/6467 [00:00<?, ?it/s]

tensor([0.60707], device='cuda:0')
tensor([39784.87109], device='cuda:0')
